# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## Section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos
import numpy as np 

# At first, we define the primary function 
def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

In [2]:
# Write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

#--------------------------------------------------------------------------------------------------------------------------------------
########################################################################################################################################
################################# EXERCISE: Compute the partial derivatives of f with respect to a,b,c #################################
########################################################################################################################################
#### As we have three independent variables (a,b,c), we need three partial derivatives
#### These partial derivatives have been calculated using Wolframalpha 
def f_prime_a(a):
  return -3*a**2 - 1/(2*np.sqrt(a))

def f_prime_b(b):
  return 2.5*b**1.5 + 3*cos(3*b)

def f_prime_c(c):
  return 1/c**2


#### Insert these partial derivatives in the gradf function 
def gradf(a, b, c):
  return [f_prime_a(a), f_prime_b(b), f_prime_c(c)]         # todo, return [df/da, df/db, df/dc]

#--------------------------------------------------------------------------------------------------------------------------------------

# Expected answer is the list of 
a,b,c = 2,3,4
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(a=a, b=b, c=c)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# Now estimate the gradient numerically without any calculus, using the approximation we used in the video.
# you should not call the function df from the last cell


#--------------------------------------------------------------------------------------------------------------------------------------
########################################################################################################################################
############################################# EXERCISE: Estimate the gradients numerically #############################################
########################################################################################################################################
numerical_grad = [0, 0, 0] # TODO

# Estimating numerically 
h = 10**-10
a_numerical_grad = (f(a+h, b, c) - f(a, b, c)) / h 
b_numerical_grad = (f(a, b+h, c) - f(a, b, c)) / h 
c_numerical_grad = (f(a, b, c+h) - f(a, b, c)) / h 
numerical_grad = [a_numerical_grad, b_numerical_grad, c_numerical_grad]

# ----------------------------------------------------------------------------------------------------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.35355817641448
OK for dim 1: expected 10.25699027111255, yours returns 10.256986371359744
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781


In [4]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

#--------------------------------------------------------------------------------------------------------------------------------------
########################################################################################################################################
############################## EXERCISE: Estimate the gradients numerically using the symmetric derivative #############################
########################################################################################################################################
numerical_grad2 = [0, 0, 0] # TODO


#### The symmetric derivative is an operation that will perform the following operation lim h->0 (f(x+h) - f(x-h) / 2h)
#### i.e. the symmetric derivative goes a small step of h to both sides of x on the function f, where we want to estimate the gradient, and then divides by 2h 
h = 10**-10
a_sym_num_grad = (f(a+h, b, c) - f(a-h, b, c)) / (2*h)
b_sym_num_grad = (f(a, b+h, c) - f(a, b-h, c)) / (2*h)
c_sym_num_grad = (f(a, b, c+h) - f(a, b, c-h)) / (2*h)
numerical_grad2 = [a_sym_num_grad, b_sym_num_grad, c_sym_num_grad]

#--------------------------------------------------------------------------------------------------------------------------------------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553735522382
OK for dim 1: expected 10.25699027111255, yours returns 10.256990812251843
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781


## section 2: support for softmax

In [36]:
# Value class starter code, with many functions taken out
from math import exp, log, log10

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)                         
    out = Value(data=self.data + other.data, _children=(self, other), _op='+')          
    
    def _backward():     
      self.grad += 1.0 * out.grad                                                       
      other.grad += 1.0 * out.grad                                                      
    out._backward = _backward                                                           
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(data=self.data * other.data, _children=(self, other), _op='*') 

    # Chain rule, local_grad * outer_grad, i.e. dz/dx = dz/dy * dy/dx 
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out


  

  #--------------------------------------------------------------------------------------------------------------------------------------
  ########################################################################################################################################
  ################################## EXERCISE: Reimplement the necessary operations to perform a softmax #################################
  ########################################################################################################################################
  ### re-implement all the other functions needed for the exercises below
  # i.e. we need the .exp() and the __truediv__ 

  # Define the __pow__ function 
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  # Define the __truediv__, which is simply a special case of the power function 
  def __truediv__(self, other): # self / other
    return self * (other**-1)

  def __rtruediv__(self, other): # other / self
    return other * self**-1

  # Define the negation function for negating it self
  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)


  # Define the exponential function
  def exp(self):
    out = Value(exp(self.data), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    return out

  # Define the logarithm function
  def log(self):
    # forward pass 
    out = Value(data=log10(self.data), _children=(self, ), _op="log")

    # backward pass 
    def _backward():
      self.grad += (1.0 / self.data) * out.grad
    out._backward = _backward
    return out 
  

  # Define the recursive add function
  def __radd__(self, other):
    return self + other

  
  
  
  #--------------------------------------------------------------------------------------------------------------------------------------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [57]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out


# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)

loss = -probs[3].log() # dim 3 acts as the label for this input example
print(f"The loss value is {loss.data}")
loss.backward()

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


The loss value is 0.944814317280191
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [66]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

# I don't know what he means here... 

logits_torch = torch.tensor([x.data for x in logits]).double(); logits_torch.requires_grad = True 
probs_torch = torch.nn.functional.softmax(input=logits_torch, dim=-1)
loss_torch = torch.nn.functional.nll_loss(input=probs_torch, target=torch.tensor([0,0,0,1]))
loss_torch.requires_grad_ = True 
loss_torch.backward()
for dim in range(4):
    print(logits_torch.grad[dim])

tensor(-0.0400, dtype=torch.float64)
tensor(0.0350, dtype=torch.float64)
tensor(0.0002, dtype=torch.float64)
tensor(0.0047, dtype=torch.float64)
